# **CAT VS DOG**

1. Download dataset (Kaggle)
2. Extract dataset
3. Import libraries
4. Load images as tf.data.Dataset
5. Create data-augmentation layers
6. Create preprocessing functions
7. Apply map → shuffle → prefetch
8. Build CNN model (anti-overfitting)
9. Compile model
10. Add EarlyStopping
11. Train the model


!nvidia-smi


In [ ]:
!nvidia-smi


Sun Dec 14 11:25:44 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 1️⃣ Install & Configure Kaggle

Installs Kaggle’s official Python API so you can download datasets directly.

In [ ]:
pip install kaggle




Stores your Kaggle API token securely

chmod 600 prevents unauthorized access

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## 2️⃣ Download & Extract Dataset

✔ Downloads the Dogs vs Cats dataset (~1GB)

In [ ]:
!kaggle datasets download -d salader/dogsvscats

Dataset URL: https://www.kaggle.com/datasets/salader/dogsvscats
License(s): unknown
 94% 1.00G/1.06G [00:07<00:01, 47.0MB/s]
100% 1.06G/1.06G [00:07<00:00, 147MB/s] 


✔ Downloads the Dogs vs Cats dataset (~1GB)

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogsvscats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

## 3️⃣ Import Libraries

✔ TensorFlow/Keras for deep learning
✔ Layers used to build CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

## STEPS:
Load images

   ↓

map() → normalize + augment

   ↓

shuffle()

   ↓

prefetch()

   ↓

model.fit()


## 4️⃣ Load Image Data (tf.data)

✔ What happens here

Automatically assigns labels:

cats → 0

dogs → 1

Converts images into batches

Creates a tf.data.Dataset

✔ Why (128,128)?

Much lower RAM usage than 256×256

Faster training

In [ ]:
train_ds=keras.utils.image_dataset_from_directory(
   directory="/content/train",
    labels='inferred',
    label_mode="int",
    batch_size=16,
    image_size=(128,128)
)

validation_ds=keras.utils.image_dataset_from_directory(
   directory="/content/test",
    labels='inferred',
    label_mode="int",
    batch_size=16,
    image_size=(128,128)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


✔ Validation data for model evaluation

## 5️⃣ Create Data-Augmentation Layers

These layers perform random transformations every batch

✔ Purpose

Artificially increases dataset size by:

Flipping images

Rotating slightly

Zooming randomly

✔ Reduces overfitting

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])


## 6️⃣ Preprocessing Functions



 Training (Normalize + Augment)

Normalization (0–255 → 0–1)

Augmentation ONLY for training

In [ ]:
def process_train(image, label):
    image = tf.cast(image, tf.float32) / 255.0   # normalize
    image = data_augmentation(image, training=True)
    return image, label


Validation (ONLY normalize)

✔ Validation must represent real data, no randomness

In [ ]:
def process_val(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


## 7️⃣ Apply map → shuffle → prefetch

| Step           | Purpose                     |
| -------------- | --------------------------- |
| `map()`        | Applies preprocessing       |
| `shuffle(500)` | Prevents memorization       |
| `prefetch(1)`  | Faster training + lower RAM |


In [ ]:
train_ds = train_ds.map(process_train)
train_ds = train_ds.shuffle(500).prefetch(1)

validation_ds = validation_ds.map(process_val)
validation_ds = validation_ds.prefetch(1)


## 8️⃣ Build CNN Model (Anti-Overfitting)

Creates a stack-based model

**Convolution Blocks**
* extract low level feature
* L2 Regularization (Weight Decay),Add it to Conv2D and Dense layers
*  Dropout (prevent overfitting)

(Same logic repeated with 64 & 128 filters)


---


**GlobalAveragePooling**


* removes Flatten()
* huge parameter reduction
* Strong anti-overfitting


---



**Dense Layers**

* Learns high-level patterns
* Dropout = strong regularization



---

**Output Layer**
Binary classification (dog vs cat)


In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import GlobalAveragePooling2D



In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),activation='relu',
                 kernel_regularizer=l2(0.001),
                 input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation='relu',
                 kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),activation='relu',
                 kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(GlobalAveragePooling2D())

model.add(Dense(64,activation='relu',
                kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,465 (400.25 KB)

 Trainable params: 102,017 (398.50 KB)

 Non-trainable params: 448 (1.75 KB)

9️⃣ Compile Model
## New Section

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

## 🔟 Early Stopping

✔ Stops training when validation loss stops improving
✔ Prevents overfitting & wasted compute

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Stops training when validation loss increases
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


## 1️⃣1️⃣ Train the Model

✔ Final training step

Uses augmented data

Evaluates on clean validation data

Early stopping protects from overfitting

In [ ]:
history=model.fit(train_ds,epochs=10,validation_data=validation_ds,callbacks=[early_stop])

Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 208s 65ms/step - accuracy: 0.5028 - loss: 0.7658 - val_accuracy: 0.5000 - val_loss: 0.6959
Epoch 2/10


## 1️⃣2️⃣ Visulalization Accuracy

Train accuracy vs validation accuracy





In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["accuracy"],color="red",label="train")
plt.plot(history.history["val_accuracy"],color="blue",label="validation")
plt.show()

## 1️⃣3️⃣Visualization Loss
Train loss vs validation loss

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history["loss"],color="red",label="train")
plt.plot(history.history["val_loss"],color="blue",label="validation")
plt.show()

## 1️⃣4️⃣TESTING

1️⃣ load_img()

In [ ]:
img = keras.utils.load_img("/content/dog.jpg", target_size=(128,128))


✔ Loads image using PIL (not OpenCV)

✔ Automatically uses RGB format

✔ Resizes image to 128×128 (same as training)

2️⃣ img_to_array()

In [ ]:
img_array = keras.utils.img_to_array(img)


✔ Converts PIL image → NumPy array
✔ Shape becomes:(128,128,3)


3️⃣ Normalize

In [ ]:
img_array = img_array / 255.0


img_array = img_array / 255.0


4️⃣ Add batch dimension

In [ ]:
img_array = np.expand_dims(img_array, axis=0)


✔ CNNs always expect batch input
✔ Shape becomes:

(1,128,128,3)



5️⃣ Predict

In [ ]:
prediction = model.predict(img_array)

6️⃣ Interpret prediction

| Output | Meaning |
| ------ | ------- |
| > 0.5  | Dog     |
| ≤ 0.5  | Cat     |

Prediction shows how sure the model is

In [ ]:
if prediction[0][0] > 0.5:
    print("Prediction: Its a DOG 🐶")
else:
    print("Prediction: Its a CAT 🐱")

print("Confidence:", prediction[0][0])